In [109]:
%load_ext tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 45280), started 1 day, 10:18:57 ago. (Use '!kill 45280' to kill it.)

In [110]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
import albumentations as A
from functools import partial
from numpy.random import default_rng
rng = default_rng()


In [111]:
SEED = 42
PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
TRAIN_DATA_FOLDER = PROJECT_FOLDER + "train_images/"
TEST_DATA_FOLDER = PROJECT_FOLDER + "test_images/"

IMAGE_SIZE = (224, 224)
input_size = 224
VAL_SPLIT = 0.1

In [112]:
train_ds = image_dataset_from_directory(
    directory=TRAIN_DATA_FOLDER,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
    subset='training',
    validation_split=VAL_SPLIT
)

val_ds = image_dataset_from_directory(
    directory=TRAIN_DATA_FOLDER,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
    subset='validation',
    validation_split=VAL_SPLIT
)

test_ds = image_dataset_from_directory(
    directory=TEST_DATA_FOLDER,
    labels=None,
    batch_size=1,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
)


Found 2081 files belonging to 100 classes.
Using 1873 files for training.
Found 2081 files belonging to 100 classes.
Using 208 files for validation.
Found 1 files belonging to 1 classes.


In [113]:
n_classes = len(train_ds.class_names)
n_classes

100

In [114]:
import random 

def augment_using_ops(images, labels):
    images = tf.image.random_flip_left_right(images)
    images = tf.image.random_brightness(images, 0.2)
    images = tf.image.random_contrast(images, 0.2, 0.5)
    return (images, labels)

train_ds_aug = train_ds.map(augment_using_ops)
val_ds_aug = train_ds.map(augment_using_ops)

In [115]:
def random_cutout(images_tensor, labels):
    ratios_x = rng.uniform(0.2, 0.6)*images_tensor.shape[1]
    ratios_y = rng.uniform(0.2, 0.6)*images_tensor.shape[2]

    xs_mask = tf.cast(ratios_x, tf.int32)
    xs_mask = xs_mask if xs_mask % 2 == 0 else xs_mask+1 # force number even

    ys_mask = tf.cast(ratios_y, tf.int32)
    ys_mask = ys_mask if ys_mask % 2 == 0 else ys_mask+1 # force number even

    return tfa.image.random_cutout(images_tensor, (xs_mask, ys_mask), constant_values = (255.0, 0.0, 0.0)), labels

train_ds_aug = train_ds_aug.map(random_cutout)
val_ds_aug = val_ds_aug.map(random_cutout)

# Models

In [116]:

def create_baseline_efficientnet_model(n_classes):
    efficientnet_layer = tf.keras.applications.EfficientNetB0(
            include_top=True,
            weights="imagenet")
    efficientnet_layer.trainable = False
    
    model = keras.Sequential([
        efficientnet_layer,
        keras.layers.Dropout(0.2),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(1000, activation='selu'),
        keras.layers.Dropout(0.2),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(1000, activation='selu'),
        keras.layers.Dropout(0.2),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-2),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model
  


In [117]:
def create_finetuned_efficientnet_model(n_classes):
    efficientnet_layer = tf.keras.applications.EfficientNetB0(
            include_top= False,
            weights="imagenet")
    efficientnet_layer.trainable = False
    
    model = keras.Sequential([
        efficientnet_layer,
        keras.layers.GlobalAveragePooling2D(name = "avg_pool"),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(n_classes, activation = 'softmax')
    ])
    
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-2),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [118]:
def create_unfreezed_efficientnet_model(model):
    for layer in model.layers[-12:]:
        if not isinstance(layer, keras.layers.BatchNormalization):
            layer.trainable = True
    
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [119]:
model = create_finetuned_efficientnet_model(n_classes)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb4 (Functional)  (None, None, None, 1792)  17673823 
                                                                 
 avg_pool (GlobalAveragePool  (None, 1792)             0         
 ing2D)                                                          
                                                                 
 batch_normalization (BatchN  (None, 1792)             7168      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 100)               179300    
                                                                 
Total params: 17,860,291
Trainable params: 182,884
Non-t

In [120]:
callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir = 'logs'
    ), 
    tf.keras.callbacks.ModelCheckpoint(
        filepath='ckp',
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
]


history = model.fit(train_ds_aug, epochs=10, callbacks=callbacks, validation_data=val_ds_aug)

59/59 [==============================] - 279s 5s/step - loss: 6.3251 - accuracy: 0.3962 - val_loss: 5.0135 - val_accuracy: 0.4709


In [121]:
# model = create_unfreezed_efficientnet_model(model)
# history = model.fit(train_ds_aug, epochs=1, callbacks=callbacks, validation_data=val_ds_aug)

In [122]:
model.evaluate(test_ds)

1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

In [123]:
model.save('empanadanet.h5')